In [16]:
import time
import numpy as np
import pandas as pd
import mgclient
from datetime import datetime
from mgclient import Node
from mgclient import Relationship
from mgclient import Path

from rasa.nlu.model import Interpreter

In [17]:
rasa_model_path = "nlu"

# create an interpreter object
interpreter = Interpreter.load(rasa_model_path)

In [18]:
def rasa_output(text):
    intent_list = []
    entity_list = []
    message = str(text).strip()
    result = interpreter.parse(message)
    
    for key in result['intent_ranking']:
        intent_list.append(key)
    for key in result['entities']:
        entity_list.append(key)
    
    #return intent_list, entity_list
    return result

Intent queries code starts here.

In [21]:
rasa_result = rasa_output("Is Anders Løland available?")
#intent_list, entity_list = rasa_output("Where can I find Pierre at 10:30?")


#List of intents
#List of probabilities of intents (floats)
#
#for key in rasa_result:
#    print(key)
print(rasa_result)

{'text': 'Is Anders Løland available?', 'intent': {'id': 7010757620765472506, 'name': 'request_person_availability', 'confidence': 0.9537244439125061}, 'entities': [{'entity': 'person', 'start': 3, 'end': 9, 'confidence_entity': 0.998601496219635, 'value': 'Anders', 'extractor': 'DIETClassifier'}], 'intent_ranking': [{'id': 7010757620765472506, 'name': 'request_person_availability', 'confidence': 0.9537244439125061}, {'id': 1198050565642826035, 'name': 'request_location_availability', 'confidence': 0.01954515650868416}, {'id': -4401168685642149560, 'name': 'affirm', 'confidence': 0.016283897683024406}, {'id': 5410358888841342465, 'name': 'move_to_location', 'confidence': 0.0046928697265684605}, {'id': -1045296936062992485, 'name': 'request_event_duration', 'confidence': 0.0018601814517751336}, {'id': 1565688475946403583, 'name': 'request_time_event', 'confidence': 0.0012640091590583324}, {'id': 989642141388351195, 'name': 'greet', 'confidence': 0.0008418871439062059}, {'id': 9165793226

In [25]:
# request_person_availability

# MENTIONED_IN

# REFERS_TO
# INFERRED_FROM
# MENTIONS
# STARTS_AT
# ENDS_AT

conn = mgclient.connect(host='127.0.0.1', port=7687)
cursor = conn.cursor()

rasa_result = rasa_output("Is Anders available at 10:30?")

for key in rasa_result:
    if (key == "intent"):
        q = ("CREATE (i:Intent {id: '%s', name: '%s', confidence: %f})"
            % (rasa_result[key]['id'], rasa_result[key]['name'], rasa_result[key]['confidence']))
        
        print(q)
        cursor.execute(q)
        
        intent_id = rasa_result[key]['id']
        
    if (key == "entities"):
        # Entites can be either:
        # Event, Location, Person, Time.
        for entity in rasa_result[key]:
            if (entity['entity'] == "event"):
                q = ("MATCH (i) WHERE i.id = '%s' CREATE (pm:EntityMention:EventMention {entity: '%s', value: '%s'})-[r:MENTIONED_IN]->(i)" 
                     % (intent_id, entity['entity'], entity['value']))
                print(q)
                cursor.execute(q)
                print("person")
                q = ("""MATCH (em: EntityMention)-[r]->(i), (e:Event)
                        WHERE em.value = "%s" AND e.name CONTAINS "%s"
                        MERGE (em)-[ref:REFERS_TO]->(e)"""
                     % (entity['value'], entity['value']))
                print(q)
                cursor.execute(q)
            if (entity['entity'] == "person"):
                entity['value'] = 'Anders Løland'
                q = ("MATCH (i) WHERE i.id = '%s' CREATE (pm:EntityMention:PersonMention {entity: '%s', value: '%s'})-[r:MENTIONED_IN]->(i)" 
                     % (intent_id, entity['entity'], entity['value']))
                print(q)
                cursor.execute(q)
                print("person")
                q = ("""MATCH (pm: EntityMention)-[r]->(i), (p:Person)
                        WHERE pm.value = "%s" AND p.name CONTAINS "%s"
                        MERGE (pm)-[ref:REFERS_TO]->(p)"""
                     % (entity['value'], entity['value']))
                print(q)
                cursor.execute(q)
            if (entity['entity'] == "time"):
                if ":" in entity['value']:
                    hour, minute = entity['value'].split(":")
                else:
                    hour = entity['value']
                    minute = 0
                unix_time = int(time.mktime(datetime.today().replace(hour=int(hour),minute=int(minute)).timetuple()))
                q = ("MATCH (i) WHERE i.id = '%s' CREATE (pm:EntityMention:TimeMention {entity: '%s', value: %d})-[r:MENTIONED_IN]->(i)" 
                     % (intent_id, entity['entity'], unix_time))
                print(q)
                cursor.execute(q)
                print("time")
                q = ("""MATCH (tm: EntityMention)-[r]->(i), (t:Time)
                        WHERE tm.value = %d AND t.time = %d
                        MERGE (tm)-[ref:REFERS_TO]->(t)"""
                     % (unix_time, unix_time))
                print(q)
                cursor.execute(q)
            if (entity['entity'] == "location"):
                q = ("MATCH (i) WHERE i.id = '%s' CREATE (pm:EntityMention:LocationMention {entity: '%s', value: '%s'})-[r:MENTIONED_IN]->(i)" 
                     % (intent_id, entity['entity'], entity['value']))
                print(q)
                cursor.execute(q)
                print("time")
                q = ("""MATCH (lm: EntityMention)-[r]->(i), (l:Location)
                        WHERE lm.value = "%s" AND l.name CONTAINS "%s"
                        MERGE (lm)-[ref:REFERS_TO]->(l)"""
                     % (entity['value'], entity['value']))
                print(q)
                cursor.execute(q)
            
            
#         q = "CREATE (i:Intent {id: '" \
#             + str(rasa_result[key]['id']) \
#             + "', name: '" \
#             + rasa_result[key]['name'] \
#             + "', confidence: '" \
#             + str(rasa_result[key]['confidence']) \
#             + "'})"
#         print("QUERY: " + q)
#         cursor.execute(q)
    
    
    
    print()
#     print(r)

q = """
    MATCH (i:Intent {name: 'request_person_availability'})-[:MENTIONS]->(pm:EntityMention {entity: 'person'})
MATCH (i)-[:MENTIONS]->(tm:EntityMention {entity: 'time'})
MATCH (pm)-[:REFERS_TO]->(p:Person {name: '%s'})<-[:HAS_PARTICIPANT]-(e:Event)
MATCH (st:Time)<-[:STARTS_AT]-(e)-[:ENDS_AT]->(et:Time)
WHERE tm.value < st.time
OR tm.value > et.time
CREATE (cr:CandiateResponse {name: 'InformPersonIsAvailable'})-[:MENTIONS]->(p)
CREATE (cr)-[:INFERRED_FROM {util: 1}]->(i)
    """
print(q)
#cursor.execute(q)
    
conn.commit()

# - event
# - location
# - person
# - time


CREATE (i:Intent {id: '7010757620765472506', name: 'request_person_availability', confidence: 0.967728})

MATCH (i) WHERE i.id = '7010757620765472506' CREATE (pm:EntityMention:PersonMention {entity: 'person', value: 'Anders Løland'})-[r:MENTIONED_IN]->(i)
person
MATCH (pm: EntityMention)-[r]->(i), (p:Person)
                        WHERE pm.value = "Anders Løland" AND p.name CONTAINS "Anders Løland"
                        MERGE (pm)-[ref:REFERS_TO]->(p)
MATCH (i) WHERE i.id = '7010757620765472506' CREATE (pm:EntityMention:TimeMention {entity: 'time', value: 1625646628})-[r:MENTIONED_IN]->(i)
time
MATCH (tm: EntityMention)-[r]->(i), (t:Time)
                        WHERE tm.value = 1625646628 AND t.time = 1625646628
                        MERGE (tm)-[ref:REFERS_TO]->(t)




    MATCH (i:Intent {name: 'request_person_availability'})-[:MENTIONS]->(pm:EntityMention {entity: 'person'})
MATCH (i)-[:MENTIONS]->(tm:EntityMention {entity: 'time'})
MATCH (pm)-[:REFERS_TO]->(p:Person {name: '%